In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [3]:
all_data = []

In [4]:
def extract_team_url_info(team_name, url):
    # Assuming the URL follows the format "https://www.transfermarkt.com/{team_url_name}/mitarbeiterhistorie/verein/{team_url_number}"
    parts = url.split('/')
    team_url_name = parts[3]
    team_url_number = parts[6]
    return team_url_name, team_url_number

In [5]:
cutoff_date = datetime(1992, 8, 15)
end_date = datetime(2024, 5, 19)  # Set this to the end of the 2023/24 season

In [6]:
team_urls = {'Manchester United':"https://www.transfermarkt.com/manchester-united/mitarbeiterhistorie/verein/985/personalie_id/0/plus/1", 
'Arsenal':"https://www.transfermarkt.com/arsenal-fc/mitarbeiterhistorie/verein/11/personalie_id/0/plus/1", 'Liverpool':"https://www.transfermarkt.com/liverpool-fc/mitarbeiterhistorie/verein/31/personalie_id/0/plus/1", 'Chelsea':"https://www.transfermarkt.com/chelsea-fc/mitarbeiterhistorie/verein/631/personalie_id/0/plus/1", 
'Tottenham Hotspur':"https://www.transfermarkt.com/tottenham-hotspur/mitarbeiterhistorie/verein/148/personalie_id/0/plus/1", 
'Manchester City':"https://www.transfermarkt.com/manchester-city/mitarbeiterhistorie/verein/281/personalie_id/0/plus/1", 'Everton':"https://www.transfermarkt.com/everton-fc/mitarbeiterhistorie/verein/29/personalie_id/0/plus/1", 
'Newcastle United':"https://www.transfermarkt.com/newcastle-united/mitarbeiterhistorie/verein/762/personalie_id/0/plus/1", 'Aston Villa':"https://www.transfermarkt.com/aston-villa/mitarbeiterhistorie/verein/405/personalie_id/0/plus/1",
'West Ham United':"https://www.transfermarkt.com/west-ham-united/mitarbeiterhistorie/verein/379/personalie_id/0/plus/1",
'Southampton':"https://www.transfermarkt.com/southampton-fc/mitarbeiterhistorie/verein/180/personalie_id/0/plus/1",
'Blackburn Rovers':"https://www.transfermarkt.com/blackburn-rovers/mitarbeiterhistorie/verein/164/personalie_id/0/plus/1",
'Leeds United':"https://www.transfermarkt.com/leeds-united/mitarbeiterhistorie/verein/399/personalie_id/0/plus/1",
'Leicester City':"https://www.transfermarkt.com/leicester-city/mitarbeiterhistorie/verein/1003/personalie_id/0/plus/1",
'Fulham':"https://www.transfermarkt.com/fulham-fc/mitarbeiterhistorie/verein/931/personalie_id/0/plus/1",
'Crystal Palace':"https://www.transfermarkt.com/crystal-palace/mitarbeiterhistorie/verein/873/personalie_id/0/plus/1",
'Middlesbrough':"https://www.transfermarkt.com/middlesbrough-fc/mitarbeiterhistorie/verein/641/personalie_id/0/plus/1",
'Sunderland':"https://www.transfermarkt.com/sunderland-afc/mitarbeiterhistorie/verein/289/personalie_id/0/plus/1",
'Bolton Wanderers':"https://www.transfermarkt.com/bolton-wanderers/mitarbeiterhistorie/verein/355/personalie_id/0/plus/1",
'West Bromwich Albion':"https://www.transfermarkt.com/west-bromwich-albion/mitarbeiterhistorie/verein/984/personalie_id/0/plus/1",
'Stoke City':"https://www.transfermarkt.com/stoke-city/mitarbeiterhistorie/verein/512/personalie_id/0/plus/1",
'Wolverhampton Wanderers':"https://www.transfermarkt.com/wolverhampton-wanderers/mitarbeiterhistorie/verein/543/personalie_id/0/plus/1",
'Sheffield Wednesday':"https://www.transfermarkt.com/sheffield-wednesday/mitarbeiterhistorie/verein/1035/personalie_id/0/plus/1",
'Coventry City':"https://www.transfermarkt.com/coventry-city/mitarbeiterhistorie/verein/990/personalie_id/0/plus/1",
'Norwich City':"https://www.transfermarkt.com/norwich-city/mitarbeiterhistorie/verein/1123/personalie_id/0/plus/1",
'Wimbledon':"https://www.transfermarkt.com/wimbledon-fc-2004-/mitarbeiterhistorie/verein/114309/personalie_id/0/plus/1",
'Charlton Athletic':"https://www.transfermarkt.com/charlton-athletic/mitarbeiterhistorie/verein/358/personalie_id/0/plus/1",
'Burnley':"https://www.transfermarkt.com/burnley-fc/mitarbeiterhistorie/verein/1132/personalie_id/0/plus/1",
'Wigan Athletic':"https://www.transfermarkt.com/wigan-athletic/mitarbeiterhistorie/verein/1071/personalie_id/0/plus/1",
'Swansea City':"https://www.transfermarkt.com/swansea-city/mitarbeiterhistorie/verein/2288/personalie_id/0/plus/1",
'Bournemouth':"https://www.transfermarkt.com/afc-bournemouth/mitarbeiterhistorie/verein/989/personalie_id/0/plus/1",
'Queens Park Rangers':"https://www.transfermarkt.com/queens-park-rangers/mitarbeiterhistorie/verein/1039/personalie_id/0/plus/1",
'Brighton & Hove Albion':"https://www.transfermarkt.com/brighton-amp-hove-albion/mitarbeiterhistorie/verein/1237/personalie_id/0/plus/1",
'Nottingham Forest':"https://www.transfermarkt.com/nottingham-forest/mitarbeiterhistorie/verein/703/personalie_id/0/plus/1",
'Portsmouth':"https://www.transfermarkt.com/portsmouth-fc/mitarbeiterhistorie/verein/1020/personalie_id/0/plus/1",
'Birmingham City':"https://www.transfermarkt.com/birmingham-city/mitarbeiterhistorie/verein/337/personalie_id/0/plus/1",
'Watford':"https://www.transfermarkt.com/watford-fc/mitarbeiterhistorie/verein/1010/personalie_id/0/plus/1",
'Derby County':"https://www.transfermarkt.com/derby-county/mitarbeiterhistorie/verein/22/personalie_id/0/plus/1",
'Ipswich Town':"https://www.transfermarkt.com/ipswich-town/mitarbeiterhistorie/verein/677/personalie_id/0/plus/1",
'Sheffield United':"https://www.transfermarkt.com/sheffield-united/mitarbeiterhistorie/verein/350/personalie_id/0/plus/1",
'Hull City':"https://www.transfermarkt.com/hull-city/mitarbeiterhistorie/verein/3008/personalie_id/0/plus/1",
'Brentford':"https://www.transfermarkt.com/brentford-fc/mitarbeiterhistorie/verein/1148/personalie_id/0/plus/1",
'Reading':"https://www.transfermarkt.com/reading-fc/mitarbeiterhistorie/verein/1032/personalie_id/0/plus/1",
'Oldham Athletic':"https://www.transfermarkt.com/oldham-athletic/mitarbeiterhistorie/verein/1078/personalie_id/0/plus/1",
'Cardiff City':"https://www.transfermarkt.com/cardiff-city/mitarbeiterhistorie/verein/603/personalie_id/0/plus/1",
'Bradford City':"https://www.transfermarkt.com/bradford-city/mitarbeiterhistorie/verein/1027/personalie_id/0/plus/1",
'Huddersfield Town':"https://www.transfermarkt.com/huddersfield-town/mitarbeiterhistorie/verein/1110/personalie_id/0/plus/1",
'Barnsley':"https://www.transfermarkt.com/barnsley-fc/mitarbeiterhistorie/verein/349/personalie_id/0/plus/1",
'Blackpool':"https://www.transfermarkt.com/blackpool-fc/mitarbeiterhistorie/verein/1181/personalie_id/0/plus/1",
'Luton Town':"https://www.transfermarkt.com/luton-town/mitarbeiterhistorie/verein/1031/personalie_id/0/plus/1",
'Swindon Town':"https://www.transfermarkt.com/swindon-town/mitarbeiterhistorie/verein/352/personalie_id/0/plus/1"}

In [7]:
def parse_date(date_string):
    try:
        return datetime.strptime(date_string, '%b %d, %Y')
    except ValueError:
        return None

In [8]:
def scrape_team_history(team_url_name, team_url_number):
    team_history_url = f"https://www.transfermarkt.com/{team_url_name}/platzierungen/verein/{team_url_number}"
    driver.get(team_history_url)
    
    # Wait until the table with team history is present
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'zentriert')))
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    rows = soup.find_all('tr', class_=['odd', 'even'])
    premier_league_seasons = []
    
    for row in rows:
        season_td = row.find('td', class_='zentriert')
        season = season_td.text.strip()
        league_td = row.find('td', class_='hauptlink').find('a')
        league_name = league_td.text.strip() if league_td else None
        
        if 'Premier League' in league_name:
            premier_league_seasons.append(season)

    return premier_league_seasons

In [9]:
def scrape_manager_history(team_name, team_url_name, team_url_number, url):
    driver.get(url)
    
    # Wait until the table with manager history is present
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'items')))
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    premier_league_seasons = scrape_team_history(team_url_name, team_url_number)
    
    if not premier_league_seasons:
        print(f"No Premier League history found for {team_name}")
        return

    table = soup.find('table', class_='items').find('tbody')
    rows = table.find_all('tr', class_=['odd', 'even'])

    for row in rows:
        name_td = row.find('td', class_='hauptlink')
        if not name_td:
            continue
        
        name = name_td.text.strip()

        zentriert_tds = row.find_all('td', class_='zentriert')
        appointed_td = zentriert_tds[1]
        appointed_date_str = appointed_td.text.strip()
        appointed_date = parse_date(appointed_date_str)

        end_td = zentriert_tds[2]
        end_date_str = end_td.text.strip() if end_td.text.strip() else 'Still in job'
        end_date = parse_date(end_date_str) if end_date_str != 'Still in job' else datetime.today()

        # Check if the manager's tenure falls within the Premier League date range
        if appointed_date and end_date:
            if appointed_date <= end_date and end_date >= cutoff_date and appointed_date <= end_date:
                all_data.append({
                    'Team': team_name,
                    'Name': name,
                    'Appointed': appointed_date_str,
                    'End of Time': end_date_str,
                    'Premier League Season': ', '.join(premier_league_seasons)
                })

In [10]:
for team_name, url in team_urls.items():
    team_url_name, team_url_number = extract_team_url_info(team_name, url)
    scrape_manager_history(team_name, team_url_name, team_url_number, url)

In [11]:
df = pd.DataFrame(all_data)
df.head()

,Team,Name,Appointed,End of Time,Premier League Season
0,Manchester United,Erik ten Hag,"Jul 1, 2022",Still in job,"24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/1..."
1,Manchester United,Ralf Rangnick,"Dec 3, 2021","May 31, 2022","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/1..."
2,Manchester United,Michael Carrick,"Nov 21, 2021","Dec 3, 2021","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/1..."
3,Manchester United,Ole Gunnar Solskjær,"Mar 28, 2019","Nov 21, 2021","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/1..."
4,Manchester United,Ole Gunnar Solskjær,"Dec 19, 2018","Mar 28, 2019","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/1..."


In [12]:
df.tail()

,Team,Name,Appointed,End of Time,Premier League Season
1386,Swindon Town,Colin Todd,"May 4, 2000","Nov 1, 2000",93/94
1387,Swindon Town,Jimmy Quinn,"Oct 2, 1998","May 2, 2000",93/94
1388,Swindon Town,Steve McMahon,"Nov 28, 1994","Sep 23, 1998",93/94
1389,Swindon Town,John Gorman,"Jul 1, 1993","Nov 21, 1994",93/94
1390,Swindon Town,Glenn Hoddle,"Apr 4, 1991","Jun 4, 1993",93/94


In [13]:
df.loc[df['Team']=='Ipswich Town']

,Team,Name,Appointed,End of Time,Premier League Season
1000,Ipswich Town,Kieran McKenna,"Dec 20, 2021",Still in job,"24/25, 01/02, 00/01, 94/95, 93/94, 92/93"
1001,Ipswich Town,John McGreal,"Dec 6, 2021","Dec 19, 2021","24/25, 01/02, 00/01, 94/95, 93/94, 92/93"
1002,Ipswich Town,Paul Cook,"Mar 2, 2021","Dec 4, 2021","24/25, 01/02, 00/01, 94/95, 93/94, 92/93"
1003,Ipswich Town,Matt Gill,"Feb 28, 2021","Mar 2, 2021","24/25, 01/02, 00/01, 94/95, 93/94, 92/93"
1004,Ipswich Town,Paul Lambert,"Oct 28, 2018","Feb 28, 2021","24/25, 01/02, 00/01, 94/95, 93/94, 92/93"
1005,Ipswich Town,Bryan Klug,"Oct 26, 2018","Oct 28, 2018","24/25, 01/02, 00/01, 94/95, 93/94, 92/93"
1006,Ipswich Town,Paul Hurst,"May 30, 2018","Oct 25, 2018","24/25, 01/02, 00/01, 94/95, 93/94, 92/93"
1007,Ipswich Town,Bryan Klug,"Apr 11, 2018","May 29, 2018","24/25, 01/02, 00/01, 94/95, 93/94, 92/93"
1008,Ipswich Town,Mick McCarthy,"Nov 1, 2012","Apr 10, 2018","24/25, 01/02, 00/01, 94/95, 93/94, 92/93"
1009,Ipswich Town,Chris Hutchings,"Oct 25, 2012","Oct 31, 2012","24/25, 01/02, 00/01, 94/95, 93/94, 92/93"


In [14]:
df.loc[df['Team']=='Wimbledon']

,Team,Name,Appointed,End of Time,Premier League Season
642,Wimbledon,Stuart Murdoch,"May 1, 2002","Jun 30, 2004","99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/9..."
643,Wimbledon,Terry Burton,"May 1, 2000","Apr 25, 2002","99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/9..."
644,Wimbledon,Egil Olsen,"Jul 1, 1999","May 1, 2000","99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/9..."
645,Wimbledon,Joe Kinnear,"Jan 19, 1992","Jun 9, 1999","99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/9..."


In [15]:
from datetime import datetime, timedelta

In [16]:
def season_to_date_range(season):
    if season == '19/20':
        season_start_date = datetime(2019, 8, 1)
        season_end_date = datetime(2020, 7, 26)
    else:
        season_start_year, season_end_year = season.split('/')
        season_start_date = datetime(int('19' + season_start_year) if int(season_start_year) >= 50 else int('20' + season_start_year), 8, 1)
        season_end_date = datetime(int('19' + season_end_year) if int(season_end_year) >= 50 else int('20' + season_end_year), 5, 31)
    return season_start_date, season_end_date

In [17]:
def is_within_range(appointed_date, end_date, season_start_date, season_end_date):
    return (appointed_date <= season_end_date and end_date >= season_start_date)

In [18]:
def filter_managers_based_on_premier_league_seasons(df):
    filtered_rows = []
    
    for index, row in df.iterrows():
        appointed_date = parse_date(row['Appointed'])
        end_date = parse_date(row['End of Time']) if row['End of Time'] != 'Still in job' else datetime.today()
        
        # Parse the Premier League seasons
        premier_league_seasons = row['Premier League Season'].split(', ')
        
        valid_tenure = False
        
        for season in premier_league_seasons:
            if season == '24/25':
                continue  # Skip the 2024/25 season
            
            season_start_date, season_end_date = season_to_date_range(season)
            
            # Check if the manager's tenure overlaps with the Premier League season range
            if is_within_range(appointed_date, end_date, season_start_date, season_end_date):
                valid_tenure = True
                break
        
        # If the manager had valid Premier League seasons, keep their row
        if valid_tenure:
            filtered_rows.append(row)
    
    # Return the filtered dataframe
    return pd.DataFrame(filtered_rows)

In [19]:
df_filtered = filter_managers_based_on_premier_league_seasons(df)

In [21]:
df_filtered.head(45)

,Team,Name,Appointed,End of Time,Premier League Season
0,Manchester United,Erik ten Hag,"Jul 1, 2022",Still in job,"24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/1..."
1,Manchester United,Ralf Rangnick,"Dec 3, 2021","May 31, 2022","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/1..."
2,Manchester United,Michael Carrick,"Nov 21, 2021","Dec 3, 2021","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/1..."
3,Manchester United,Ole Gunnar Solskjær,"Mar 28, 2019","Nov 21, 2021","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/1..."
4,Manchester United,Ole Gunnar Solskjær,"Dec 19, 2018","Mar 28, 2019","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/1..."
5,Manchester United,José Mourinho,"Jul 1, 2016","Dec 18, 2018","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/1..."
6,Manchester United,Louis van Gaal,"Jul 14, 2014","May 23, 2016","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/1..."
7,Manchester United,Ryan Giggs,"Apr 23, 2014","Jun 30, 2014","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/1..."
8,Manchester United,David Moyes,"Jul 1, 2013","Apr 22, 2014","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/1..."
9,Manchester United,Sir Alex Ferguson,"Nov 6, 1986","Jun 30, 2013","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/1..."


In [22]:
df_filtered.loc[df_filtered['Team']=='Ipswich Town']

,Team,Name,Appointed,End of Time,Premier League Season
1016,Ipswich Town,George Burley,"Dec 28, 1994","Oct 11, 2002","24/25, 01/02, 00/01, 94/95, 93/94, 92/93"
1017,Ipswich Town,Paul Goddard,"Dec 5, 1994","Dec 28, 1994","24/25, 01/02, 00/01, 94/95, 93/94, 92/93"
1018,Ipswich Town,John Lyall,"May 11, 1990","Dec 5, 1994","24/25, 01/02, 00/01, 94/95, 93/94, 92/93"


In [23]:
df_filtered.loc[df_filtered['Team']=='Luton Town']

,Team,Name,Appointed,End of Time,Premier League Season
1323,Luton Town,Rob Edwards,"Nov 17, 2022",Still in job,23/24


In [24]:
df_filtered.loc[df_filtered['Team']=='Burnley']

,Team,Name,Appointed,End of Time,Premier League Season
676,Burnley,Vincent Kompany,"Jul 1, 2022","Jun 30, 2024","23/24, 21/22, 20/21, 19/20, 18/19, 17/18, 16/1..."
677,Burnley,Mike Jackson,"Apr 15, 2022","Jun 30, 2022","23/24, 21/22, 20/21, 19/20, 18/19, 17/18, 16/1..."
678,Burnley,Sean Dyche,"Oct 30, 2012","Apr 15, 2022","23/24, 21/22, 20/21, 19/20, 18/19, 17/18, 16/1..."
682,Burnley,Brian Laws,"Jan 13, 2010","Dec 29, 2010","23/24, 21/22, 20/21, 19/20, 18/19, 17/18, 16/1..."
683,Burnley,Martin Dobson,"Jan 10, 2010","Jan 13, 2010","23/24, 21/22, 20/21, 19/20, 18/19, 17/18, 16/1..."
684,Burnley,Owen Coyle,"Nov 22, 2007","Jan 8, 2010","23/24, 21/22, 20/21, 19/20, 18/19, 17/18, 16/1..."


In [27]:
pd.set_option("max_colwidth", None)
print(df_filtered.loc[df_filtered['Team']=='Burnley', 'Premier League Season'])

676    23/24, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 14/15, 09/10
677    23/24, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 14/15, 09/10
678    23/24, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 14/15, 09/10
682    23/24, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 14/15, 09/10
683    23/24, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 14/15, 09/10
684    23/24, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 14/15, 09/10
Name: Premier League Season, dtype: object


In [29]:
df_filtered.shape

(485, 5)

In [30]:
print(df_filtered.loc[df_filtered['Team']=='Arsenal', 'Premier League Season'])

10    24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 01/02, 00/01, 99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93
11    24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 01/02, 00/01, 99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93
12    24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 01/02, 00/01, 99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93
13    24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 01/02, 00/01, 99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93
14    24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 1

In [31]:
df_filtered.to_csv('EPL Managers Only.csv')

In [32]:
df_filtered.loc[df_filtered['Team']=='Everton']

,Team,Name,Appointed,End of Time,Premier League Season
98,Everton,Sean Dyche,"Jan 30, 2023",Still in job,"24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 01/02, 00/01, 99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93"
99,Everton,Leighton Baines,"Jan 24, 2023","Jan 29, 2023","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 01/02, 00/01, 99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93"
100,Everton,Paul Tait,"Jan 24, 2023","Jan 29, 2023","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 01/02, 00/01, 99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93"
101,Everton,Frank Lampard,"Jan 31, 2022","Jan 23, 2023","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 01/02, 00/01, 99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93"
102,Everton,Duncan Ferguson,"Jan 18, 2022","Jan 30, 2022","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 01/02, 00/01, 99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93"
103,Everton,Rafael Benítez,"Jul 1, 2021","Jan 16, 2022","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 01/02, 00/01, 99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93"
104,Everton,Carlo Ancelotti,"Dec 22, 2019","Jun 30, 2021","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 01/02, 00/01, 99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93"
105,Everton,Duncan Ferguson,"Dec 5, 2019","Dec 21, 2019","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 01/02, 00/01, 99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93"
106,Everton,Marco Silva,"May 31, 2018","Dec 5, 2019","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 01/02, 00/01, 99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93"
107,Everton,Sam Allardyce,"Nov 30, 2017","May 16, 2018","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 01/02, 00/01, 99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93"


In [33]:
df_filtered.loc[df_filtered['Team']=='Wimbledon']

,Team,Name,Appointed,End of Time,Premier League Season
643,Wimbledon,Terry Burton,"May 1, 2000","Apr 25, 2002","99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93"
644,Wimbledon,Egil Olsen,"Jul 1, 1999","May 1, 2000","99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93"
645,Wimbledon,Joe Kinnear,"Jan 19, 1992","Jun 9, 1999","99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93"


In [34]:
df_filtered.loc[df_filtered['Team']=='Chelsea']

,Team,Name,Appointed,End of Time,Premier League Season
29,Chelsea,Mauricio Pochettino,"Jul 1, 2023","Jun 30, 2024","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 01/02, 00/01, 99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93"
30,Chelsea,Frank Lampard,"Apr 6, 2023","Jun 30, 2023","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 01/02, 00/01, 99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93"
31,Chelsea,Bruno Saltor,"Apr 3, 2023","Apr 5, 2023","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 01/02, 00/01, 99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93"
32,Chelsea,Graham Potter,"Sep 8, 2022","Apr 2, 2023","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 01/02, 00/01, 99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93"
33,Chelsea,Thomas Tuchel,"Jan 26, 2021","Sep 7, 2022","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 01/02, 00/01, 99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93"
34,Chelsea,Frank Lampard,"Jul 4, 2019","Jan 25, 2021","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 01/02, 00/01, 99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93"
35,Chelsea,Maurizio Sarri,"Jul 14, 2018","Jun 30, 2019","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 01/02, 00/01, 99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93"
36,Chelsea,Antonio Conte,"Jul 1, 2016","Jul 13, 2018","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 01/02, 00/01, 99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93"
37,Chelsea,Guus Hiddink,"Dec 21, 2015","Jun 30, 2016","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 01/02, 00/01, 99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93"
38,Chelsea,José Mourinho,"Jul 1, 2013","Dec 17, 2015","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 01/02, 00/01, 99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93"


In [35]:
df_filtered.loc[df_filtered['Team']=='Tottenham Hotspur']

,Team,Name,Appointed,End of Time,Premier League Season
55,Tottenham Hotspur,Ange Postecoglou,"Jul 1, 2023",Still in job,"24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 01/02, 00/01, 99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93"
56,Tottenham Hotspur,Ryan Mason,"Apr 24, 2023","Jun 30, 2023","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 01/02, 00/01, 99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93"
57,Tottenham Hotspur,Cristian Stellini,"Mar 26, 2023","Apr 24, 2023","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 01/02, 00/01, 99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93"
58,Tottenham Hotspur,Antonio Conte,"Nov 2, 2021","Mar 26, 2023","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 01/02, 00/01, 99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93"
59,Tottenham Hotspur,Nuno Espírito Santo,"Jul 1, 2021","Nov 1, 2021","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 01/02, 00/01, 99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93"
60,Tottenham Hotspur,Ryan Mason,"Apr 20, 2021","Jun 30, 2021","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 01/02, 00/01, 99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93"
61,Tottenham Hotspur,José Mourinho,"Nov 20, 2019","Apr 19, 2021","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 01/02, 00/01, 99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93"
62,Tottenham Hotspur,Mauricio Pochettino,"Jul 1, 2014","Nov 19, 2019","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 01/02, 00/01, 99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93"
63,Tottenham Hotspur,Tim Sherwood,"Dec 23, 2013","May 13, 2014","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 01/02, 00/01, 99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93"
64,Tottenham Hotspur,Tim Sherwood,"Dec 17, 2013","Dec 23, 2013","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 01/02, 00/01, 99/00, 98/99, 97/98, 96/97, 95/96, 94/95, 93/94, 92/93"


In [36]:
df_filtered.loc[df_filtered['Team']=='Manchester City']

,Team,Name,Appointed,End of Time,Premier League Season
81,Manchester City,Pep Guardiola,"Jul 1, 2016",Still in job,"24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 00/01, 95/96, 94/95, 93/94, 92/93"
82,Manchester City,Manuel Pellegrini,"Jul 1, 2013","Jun 30, 2016","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 00/01, 95/96, 94/95, 93/94, 92/93"
83,Manchester City,Brian Kidd,"May 13, 2013","May 19, 2013","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 00/01, 95/96, 94/95, 93/94, 92/93"
84,Manchester City,Roberto Mancini,"Dec 19, 2009","May 13, 2013","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 00/01, 95/96, 94/95, 93/94, 92/93"
85,Manchester City,Mark Hughes,"Jun 4, 2008","Dec 19, 2009","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 00/01, 95/96, 94/95, 93/94, 92/93"
86,Manchester City,Sven-Göran Eriksson,"Jul 6, 2007","Jun 2, 2008","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 00/01, 95/96, 94/95, 93/94, 92/93"
87,Manchester City,Stuart Pearce,"Mar 11, 2005","May 14, 2007","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 00/01, 95/96, 94/95, 93/94, 92/93"
88,Manchester City,Kevin Keegan,"May 24, 2001","Mar 10, 2005","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 00/01, 95/96, 94/95, 93/94, 92/93"
89,Manchester City,Joe Royle,"Feb 18, 1998","May 21, 2001","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 00/01, 95/96, 94/95, 93/94, 92/93"
94,Manchester City,Alan Ball,"Jul 2, 1995","Aug 27, 1996","24/25, 23/24, 22/23, 21/22, 20/21, 19/20, 18/19, 17/18, 16/17, 15/16, 14/15, 13/14, 12/13, 11/12, 10/11, 09/10, 08/09, 07/08, 06/07, 05/06, 04/05, 03/04, 02/03, 00/01, 95/96, 94/95, 93/94, 92/93"


In [37]:
df_filtered.tail()

,Team,Name,Appointed,End of Time,Premier League Season
1231,Huddersfield Town,David Wagner,"Nov 9, 2015","Jan 14, 2019","18/19, 17/18"
1291,Barnsley,Danny Wilson,"Jun 2, 1994","Jul 5, 1998",97/98
1313,Blackpool,Ian Holloway,"Jul 1, 2009","Nov 3, 2012",10/11
1323,Luton Town,Rob Edwards,"Nov 17, 2022",Still in job,23/24
1389,Swindon Town,John Gorman,"Jul 1, 1993","Nov 21, 1994",93/94


In [38]:
df_filtered.loc[df_filtered['Team']=='Ipswich Town']

,Team,Name,Appointed,End of Time,Premier League Season
1016,Ipswich Town,George Burley,"Dec 28, 1994","Oct 11, 2002","24/25, 01/02, 00/01, 94/95, 93/94, 92/93"
1017,Ipswich Town,Paul Goddard,"Dec 5, 1994","Dec 28, 1994","24/25, 01/02, 00/01, 94/95, 93/94, 92/93"
1018,Ipswich Town,John Lyall,"May 11, 1990","Dec 5, 1994","24/25, 01/02, 00/01, 94/95, 93/94, 92/93"


In [39]:
df_filtered.shape

(485, 5)